In [12]:
!pip install onnx neo4j-driver

In [ ]:
from onnx.onnx_pb2 import *
from onnx import checker, helper

In [2]:
# graph
graph = helper.make_graph(
    [
        helper.make_node("FC", ["X", "W1", "B1"], ["H1"]),
        helper.make_node("Relu", ["H1"], ["R1"]),
        helper.make_node("FC", ["R1", "W2", "B2"], ["Y"]),
    ],
    "MLP",
    ["X", "W1", "B1", "W2", "B2"],
    ["Y"]
)

print(str(graph))


node {
  input: "X"
  input: "W1"
  input: "B1"
  output: "H1"
  op_type: "FC"
}
node {
  input: "H1"
  output: "R1"
  op_type: "Relu"
}
node {
  input: "R1"
  input: "W2"
  input: "B2"
  output: "Y"
  op_type: "FC"
}
name: "MLP"
input: "X"
input: "W1"
input: "B1"
input: "W2"
input: "B2"
output: "Y"
ir_version: 1



In [ ]:
from neo4j.v1 import GraphDatabase, basic_auth

driver = GraphDatabase.driver("bolt://localhost:7687", auth=basic_auth("neo4j", "test"))
session = driver.session()

In [10]:
statement = """	
MERGE (g:Graph {name:$graph.name}) SET g += $graph.attr
FOREACH (name in $graph.inputs | MERGE (i:Node {name:name})-[:INPUT]->(g) SET i:Input)
FOREACH (name in $graph.outputs | MERGE (g)<-[:OUTPUT]-(o:Node {name:name}) SET o:Output)
WITH *
UNWIND $graph.nodes as node
CREATE (n:Node) SET n += node.attr
FOREACH (name in node.inputs | MERGE (i:Node {name:name}) SET i:Input MERGE (i)-[:IN]->(n))
FOREACH (name in node.outputs | MERGE (o:Node {name:name}) SET o:Output MERGE (n)-[:OUT]->(o));
"""

params = {"graph":{"name":graph.name, "attr":{"ir_version":graph.ir_version}, 
                   "inputs":graph.input._values, "outputs":graph.output._values,
                   "nodes": list(map(lambda n : {"attr":{"op_type":n.op_type},
                         "inputs":n.input._values,"outputs":n.output._values}, graph.node))}}
print(params)

{'graph': {'name': 'MLP', 'attr': {'ir_version': 1}, 'inputs': ['X', 'W1', 'B1', 'W2', 'B2'], 'outputs': ['Y'], 'nodes': [{'attr': {'op_type': 'FC'}, 'inputs': ['X', 'W1', 'B1'], 'outputs': ['H1']}, {'attr': {'op_type': 'Relu'}, 'inputs': ['H1'], 'outputs': ['R1']}, {'attr': {'op_type': 'FC'}, 'inputs': ['R1', 'W2', 'B2'], 'outputs': ['Y']}]}}


In [11]:
summary = session.run(statement,params).summary
print(summary)
print(vars(summary))

<bound method StatementResult.summary of <neo4j.v1.result.BoltStatementResult object at 0x10850d8d0>>
{}


In [ ]:
session.close()